In [1]:
# First step in creating program is to extract the relevant infromation from a PDF table 

from tika import parser 
import pandas as pd 
import os
import re
#from pathlib import Path

In [4]:
def extract(pdf_path):
    """
    This function will parse the PDF into a string and from there it will extract 
    out gps coordiantes from the document
    """
    # opening up PDF with tika parser
    parsed_pdf = parser.from_file(pdf_path)
    
    # saving content of PDF
    pdf_data = parsed_pdf['content'] 

    # get rid of all white space from parsed PDF
    cleaned_data = "".join(pdf_data.split())

    # pattern for gps coordinates
    pattern = re.compile('''(\d{1,3}°\d{1,3}′\d{1,3}.\d{1,3}′′[A-Za-z],\d{1,3}°\d{1,3}′\d{1,3}.\d{1,3}′′[A-Za-z])''')  

    # match the pattern to the parsed data.
    gps_coords = pattern.findall(cleaned_data)
    
    # split list into latitude and longitude
    split_coords = []

    for elem in gps_coords:
        lst = elem.split(",")
        lst2 = [lst[0], lst[1]]
        split_coords.append(lst2)

    # convert dictionary to data frame 
    coords_df = pd.DataFrame(split_coords,
                   columns=['Latitude', 'Longitude'])
    df['Lat Hours'] = coords_df['Latitude'].str.split("°").str[0]
    
    return(coords_df)

In [5]:
extract(os.path.realpath("../project/documents/MurphyRTL2017.pdf"))

2021-03-29 22:44:05,346 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


AttributeError: 'PandasArray' object has no attribute '_str_split'